# Large_Scale_Analysis.ipynb
This notebook:
* Combines the output from `english-fisher-annotator` as it is in 2 parts (due to running the two parts in parallel across 2 GPUs).
* Obtains counts for the tokens and disfluent nodes.

In [1]:
import utils_general

import os
import difflib
import re

import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
df_GoogleASR_0 = pd.read_csv("./csv/df-english-fisher-annotations-train-GoogleASR-0.csv", index_col=False)
df_GoogleASR_1 = pd.read_csv("./csv/df-english-fisher-annotations-train-GoogleASR-1.csv", index_col=False)
df_GoogleASR = pd.concat([df_GoogleASR_0, df_GoogleASR_1], ignore_index=True)

df_WhisperX_0 = pd.read_csv("./csv/df-english-fisher-annotations-train-WhisperX-0.csv", index_col=False)
df_WhisperX_1 = pd.read_csv("./csv/df-english-fisher-annotations-train-WhisperX-1.csv", index_col=False)
df_WhisperX = pd.concat([df_WhisperX_0, df_WhisperX_1], ignore_index=True)

rename_dict = {"parse": "WhisperX_parse", 
               "orig_dys": "WhisperX_orig_dys", 
               "dys": "WhisperX_dys",
               "transcript": "WhisperX"}
df_WhisperX = df_WhisperX.rename(columns=rename_dict)

# keep all cols in the merge
df = None
common_columns = ["show_uri", "show_name", "show_description", "publisher", "rss_link", "episode_uri", "episode_name", "episode_description", "duration", "show_filename_prefix", "episode_filename_prefix", "language", "transcript_length", "WhisperX"]
df = pd.merge(df_GoogleASR, df_WhisperX, on=common_columns, how="outer")
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
display(df)

# save df
df.to_csv("./csv/df-english-fisher-annotations-train-all2.csv", header=True)

,show_uri,show_name,show_description,publisher,rss_link,episode_uri,episode_name,episode_description,duration,show_filename_prefix,episode_filename_prefix,language,WhisperX,transcript_length,GoogleASR,GoogleASR_parse,GoogleASR_orig_dys,GoogleASR_dys,WhisperX_parse,WhisperX_orig_dys,WhisperX_dys
0,spotify:show:2NYtxEZyYelR6RMKmjfPLB,Kream in your Koffee,A 20-something blunt female takes on the world...,Katie Houle,https://anchor.fm/s/11b84b68/podcast/rss,spotify:episode:000A9sRBYdVh66csG2qEdj,1: It’s Christmas Time!,On the first ever episode of Kream in your Kof...,12.700133,show_2NYtxEZyYelR6RMKmjfPLB,000A9sRBYdVh66csG2qEdj,en,"Hello, hello, hello everyone. This is Katie an...",1716,Hello. Hello. Hello everyone. This is Katie an...,(INTJ (UNK hello))\n(INTJ (UNK hello))\n(FRAG ...,hello _\nhello _\nhello _ everyone _\nthis _ i...,hello _ hello _ hello E everyone _ this _ is _...,(FRAG (INTJ (UNK hello)) (INTJ (UNK hello)) (I...,hello _ hello _ hello _ everyone _\nthis _ is ...,hello E hello E hello E everyone _ this _ is _...
1,spotify:show:6vZRgUFTYwbAA79UNCADr4,Inside The 18 : A Podcast for Goalkeepers by G...,Inside the 18 is your source for all things Go...,Inside the 18 GK Media,https://anchor.fm/s/81a072c/podcast/rss,spotify:episode:001UfOruzkA3Bn1SPjcdfa,Ep.36 - Incorporating a Singular Goalkeeping C...,Today’s episode is a sit down Michael and Omar...,43.616333,show_6vZRgUFTYwbAA79UNCADr4,001UfOruzkA3Bn1SPjcdfa,en,Welcome to Inside the 18. Today's episode is a...,2017,Welcome to inside the 18. Today's episode is t...,(ADJP (UNK welcome) (PP (UNK to) (PP (UNK insi...,welcome _ to _ inside _ the _\ntoday _ 's _ ep...,welcome _ to _ inside _ the _ today _ 's _ epi...,(ADJP (UNK welcome) (PP (UNK to) (PP (UNK insi...,welcome _ to _ inside _ the _\ntoday _ 's _ ep...,welcome _ to _ inside _ the _ today _ 's _ epi...
2,spotify:show:5BvKEjaMSuvUsGROGi2S7s,Arrowhead Live!,Your favorite podcast for everything @Chiefs! ...,Arrowhead Live!,https://anchor.fm/s/917dba4/podcast/rss,spotify:episode:001i89SvIQgDuuyC53hfBm,Episode 1: Arrowhead Live! Debut,Join us as we take a look at all current Chief...,58.189200,show_5BvKEjaMSuvUsGROGi2S7s,001i89SvIQgDuuyC53hfBm,en,"Hey Cheese fans! Before we get started, I want...",1518,Hey cheese fans before we get started. I wante...,(FRAG (INTJ (UNK hey)) (NP (UNK cheese) (UNK f...,hey _ cheese _ fans _ before _ we _ get _ star...,hey E cheese _ fans _ before _ we _ get _ star...,(FRAG (INTJ (UNK hey)) (NP (UNK cheese) (UNK f...,hey _ cheese _ fans _\nbefore _ we _ get _ sta...,hey E cheese _ fans _ before _ we _ get _ star...
3,spotify:show:7w3h3umpH74veEJcbE6xf4,FBoL,"The comedy podcast about toxic characters, wri...",Emily Edwards,https://www.fuckboisoflit.com/episodes?format=rss,spotify:episode:0025RWNwe2lnp6HcnfzwzG,"The Lion, The Witch, And The Wardrobe - Ashley...",The modern morality tail of how to stay good f...,51.782050,show_7w3h3umpH74veEJcbE6xf4,0025RWNwe2lnp6HcnfzwzG,en,"Sorry to interrupt the show, but I do have to ...",1707,"Sorry to interrupt the show, but I do have to ...",(S (ADJP (UNK sorry) (S (VP (UNK to) (VP (UNK ...,sorry _ to _ interrupt _ the _ show _ but _ i ...,sorry _ to _ interrupt _ the _ show _ but _ i ...,(S (ADJP (UNK sorry) (S (VP (UNK to) (VP (UNK ...,sorry _ to _ interrupt _ the _ show _ but _ i ...,sorry _ to _ interrupt _ the _ show _ but _ i ...
4,spotify:show:5ljREb8VLogQLT7AKGwav1,UPSC Podcasts,Podcasts useful for UPSC aspirants! Mainly dis...,UPSC Podcast,https://anchor.fm/s/8afceec/podcast/rss,spotify:episode:0025w0gdgkl11Nzkmg1wnm,Tourism in India : Opportunities and Challenges,.,13.788000,show_5ljREb8VLogQLT7AKGwav1,0025w0gdgkl11Nzkmg1wnm,en,This is All India Radio. In the program Spotli...,1755,This is all India radio in the program Spotlig...,(S (NP (UNK this)) (VP (UNK is) (NP (UNK all) ...,this _ is _ all _ india _ radio _ in _ the _ p...,this _ is _ all _ india _ radio _ in _ the _ p...,(S (NP (UNK this)) (VP (UNK is) (NP (UNK all) ...,this _ is 

In [3]:
# uh, um counts
for target_str in ["uh", "um"]:
        for asr_str in ["GoogleASR", "WhisperX"]:
            df[f"{target_str}_count_{asr_str}"] = df[f"{asr_str}"].str.count(r"\b"+re.escape(target_str)+r"\b", flags=re.IGNORECASE)
            
# english-fisher-annotator parse counts
for target in ["INTJ", "PRN", "EDITED"]:
    for asr in ["WhisperX", "GoogleASR"]:
        df[f"{target}_count_{asr}"] = -1
        df[f"{target}_count_{asr}"] = df[f"{asr}_parse"].str.count(r"\("+re.escape(target)+r" ", flags=re.IGNORECASE)
            
display(df.describe().round(2))

,duration,transcript_length,uh_count_GoogleASR,uh_count_WhisperX,um_count_GoogleASR,um_count_WhisperX,INTJ_count_WhisperX,INTJ_count_GoogleASR,PRN_count_WhisperX,PRN_count_GoogleASR,EDITED_count_WhisperX,EDITED_count_GoogleASR
count,82601.00,82601.00,82601.00,82601.00,82601.00,82601.00,82601.00,82601.00,82601.00,82601.00,82601.00,82601.00
mean,40.44,1682.65,0.09,1.38,0.25,1.69,50.90,48.02,10.84,12.71,16.71,30.26
std,20.10,267.51,0.35,3.03,0.70,3.14,39.88,37.12,9.79,11.29,9.58,13.71
min,10.00,18.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,23.72,1530.00,0.00,0.00,0.00,0.00,19.00,18.00,4.00,5.00,10.00,20.00
50%,38.61,1702.00,0.00,0.00,0.00,0.00,42.00,39.00,9.00,10.00,15.00,30.00
75%,54.85,1859.00,0.00,2.00,0.00,2.00,74.00,70.00,15.00,18.00,22.00,39.00
max,304.95,2650.00,10.00,110.00,18.00,119.00,310.00,308.00,169.00,187.00,119.00,148.00
